In [ ]:
Part №2

In [2]:
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Input, Flatten, Dense
from tensorflow.keras.models import Model

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

In [3]:
# Завантаження згорткової основи VGG16
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
base_model.trainable = False

x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dense(10, activation='softmax')(x)

# Створення моделі
model = Model(inputs=base_model.input, outputs=x)

# Компіляція моделі
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])



58889256/58889256 [==============================] - 4s 0us/step


In [4]:
x_train_resized = tf.image.resize(x_train, (32, 32))
x_test_resized = tf.image.resize(x_test, (32, 32))

x_train_resized = tf.tile(x_train_resized, [1, 1, 1, 3])
x_test_resized = tf.tile(x_test_resized, [1, 1, 1, 3])

# Навчання моделі
model.fit(x_train_resized, y_train, batch_size=128, epochs=10, validation_data=(x_test_resized, y_test))

Epoch 1/10



469/469 [==============================] - 241s 511ms/step - loss: 0.5715 - accuracy: 0.7999 - val_loss: 0.4612 - val_accuracy: 0.8332
Epoch 2/10
469/469 [==============================] - 288s 614ms/step - loss: 0.4223 - accuracy: 0.8453 - val_loss: 0.4307 - val_accuracy: 0.8473
Epoch 3/10
469/469 [==============================] - 298s 637ms/step - loss: 0.3870 - accuracy: 0.8574 - val_loss: 0.4049 - val_accuracy: 0.8557
Epoch 4/10
469/469 [==============================] - 289s 616ms/step - loss: 0.3644 - accuracy: 0.8653 - val_loss: 0.4002 - val_accuracy: 0.8580
Epoch 5/10
469/469 [==============================] - 290s 618ms/step - loss: 0.3488 - accuracy: 0.8725 - val_loss: 0.3799 - val_accuracy: 0.8664
Epoch 6/10
469/469 [==============================] - 303s 646ms/step - loss: 0.3354 - accuracy: 0.8763 - val_loss: 0.3810 - val_accuracy: 0.8650
Epoch 7/10
469/469 [==============================] - 313s 668ms/step - loss: 0.3227 - accuracy: 0.8793 - val_loss: 0.3769 - val_accura

In [5]:
# Оцінка моделі на тестових даних
test_loss, test_accuracy = model.evaluate(x_test_resized, y_test, verbose=0)
print(f"Test Accuracy: {test_accuracy}")

Test Accuracy: 0.8734999895095825


In [6]:
# Донавчання останніх трьох шарів
for i, layer in enumerate(model.layers):
    layer._name = 'layer_' + str(i)

base_model.trainable = True
set_trainable = False
for layer in base_model.layers:
    if layer.name == "block5_conv1":
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

modified_model = Sequential([
   base_model,
   Flatten(),
   Dense(256, activation="relu"),
   Dense(10, activation="softmax"),
])

# Компіляція моделі
modified_model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Зміна розміру вихідних даних до (32, 32)
x_train_resized = tf.image.resize(x_train, (32, 32))

# Додавання додаткових каналів
x_train_resized = tf.tile(x_train_resized, [1, 1, 1, 3])

# Навчання моделі
history_modified = modified_model.fit(
    x_train_resized,
    y_train,
    epochs=5,
    verbose=1,
    batch_size=64,
    validation_split=0.2
)

Epoch 1/5
750/750 [==============================] - 224s 297ms/step - loss: 0.9834 - accuracy: 0.7072 - val_loss: 0.6479 - val_accuracy: 0.7843
Epoch 2/5
750/750 [==============================] - 237s 316ms/step - loss: 0.5805 - accuracy: 0.8016 - val_loss: 0.5514 - val_accuracy: 0.8076
Epoch 3/5
750/750 [==============================] - 240s 320ms/step - loss: 0.5122 - accuracy: 0.8209 - val_loss: 0.5088 - val_accuracy: 0.8207
Epoch 4/5
750/750 [==============================] - 240s 321ms/step - loss: 0.4750 - accuracy: 0.8324 - val_loss: 0.4810 - val_accuracy: 0.8270
Epoch 5/5
750/750 [==============================] - 239s 318ms/step - loss: 0.4516 - accuracy: 0.8393 - val_loss: 0.4611 - val_accuracy: 0.8330


In [7]:
#Оцінка результатів
test_loss, test_accuracy = modified_model.evaluate(x_test_resized, y_test, verbose=0)
print(f"Test Accuracy: {test_accuracy}")

Test Accuracy: 0.8373000025749207


In [11]:
Вивід : Порівнюючи звичайну мережу згорткової з мережею згортання, що використовує VGG16 в якості основи, можемо помітити, що звичайна мережа згортання показала більш високу точність. 
Донавчання не дало підвищення точності моделі.